In [1]:
# Specify location to FlavorPy files
directory_to_git_folder = "/home/alex/GitRepos/FlavorPy/flavorpy"
import os
os.chdir(os.path.expanduser(directory_to_git_folder))

import modelfitting as mf
import numpy as np
import pandas as pd

# Simple example

This example is supposed to serve as a quick start to the ModelFitting module of FlavorPy.

## Import FlavorPy

After installing FlavorPy with 'pip install flavorpy', import the modelfitting module.

In [ ]:
# Import the modelfitting module of FlavorPy
import flavorpy.modelfitting as mf

# We will also need numpy and pandas
import numpy as np
import pandas as pd

## Defining mass matrices

To define a model of leptons, we start by defining its mass matrices

$m_e = \begin{pmatrix}v_1 & v_2 & v_3 \\ v_3 & v_1 & v_2 \\ v_2 & v_3 & v_1\end{pmatrix} \quad$ and
$\quad m_n = \begin{pmatrix}v_1 & v_2 & v_3 \\ v_2 & v_1 & 2 \\ v_3 & 2 & v_1\end{pmatrix}$

For this example we have:

In [2]:
# Charged lepton mass matrix
def Me(params):
    v1, v2, v3 = params['v1'], params['v2'], params['v3']
    return np.array([[v1, v2 ,v3], [v3, v1, v2], [v2, v3, v1]])

# Neutrino mass matrix
def Mn(params):
    v1, v2, v3 = params['v1'], params['v2'], params['v3']
    return np.array([[v1, v2, v3], [v2, v1, 2], [v3, 2, v1]])

## Defining parameterspace

Next, we define the parameterspace of our model. We therefore construct an empty parameter space and add the parameters to it. When drawing random points in our parameter space, we will evaluate the 'sample_fct', which in this case is numpys uniform distribution between 0 and 1.

In [3]:
ParamSpace = mf.ParameterSpace()
ParamSpace.add_dim(name='v1', sample_fct=np.random.uniform)
ParamSpace.add_dim(name='v2', sample_fct=np.random.uniform)
ParamSpace.add_dim(name='v3', sample_fct=np.random.uniform)

## Constructing Model

Then we can construct the lepton model as follows:

In [4]:
Model0 = mf.FlavorModel(mass_matrix_e=Me, mass_matrix_n=Mn, parameterspace=ParamSpace, ordering='NO')

Now we can determine the masses and mixing observables of a given point in parameter space by:

In [5]:
Model0.get_obs({'v1': 1.5, 'v2': 1.1, 'v3': 1.3})

{'me/mu': 0.9999999999999992,
 'mu/mt': 0.08882311833686549,
 's12^2': 0.6420066494742026,
 's13^2': 0.008093453559868123,
 's23^2': 0.0012798653500391615,
 'd/pi': 0.0,
 'r': 0.0015714838018330382,
 'm21^2': 3.901985494559771e-05,
 'm3l^2': 0.024829944094927017,
 'm1': 0.018287792823374144,
 'm2': 0.01932519653965394,
 'm3': 0.15863287005308097,
 'eta1': 1.0,
 'eta2': 0.0,
 'J': 0.0,
 'Jmax': 0.001529498244076698,
 'Sum(m_i)': 0.19624585941610906,
 'm_b': 0.023676305208819276,
 'm_bb': 0.02008529388120004,
 'nscale': 0.03548228305985795}

Here, 'me/mu' is the mass ratio of electron mass divided by muon mass, 'sij^2' refers to the mixing angles $\sin^2(\theta_{ij})$, 'd/pi' is the cp violating phase in the PMNS matrix divided by $\pi$, 'm21^2' and 'm3l^2' and the squared neutrino mass differences, i.e. mij^2 = m_i^2 - m_j^2, 'r' is their quotient r = m21^2 / m3l^2, 'm1' and 'm2' and 'm3' are the neutrino masses, 'eta1' and 'eta2' are the majorana phases, 'J' is the Jarskog determinant, 'm_b' and 'm_bb' are the effective neutrino masses for beta decay and neutrinoless double beta decay, respectively.

## Fitting model to experimental data

Let us now fit this model to a specific experimental data set. As a default the NuFit v5.2 for NO with SK data is used. To fit this model we choose for example 3 randomly drawn points in the parameter space and apply minimization algorithms to these points, in order to find a point that matches the experimental data well. Note that by default 4 minimization algorithms are applied consecutively to all 3 random points such that we get 12 points in the end.

In [6]:
pd.set_option('display.max_columns', None)  # This pandas setting allows us to see all columns

df = Model0.make_fit(points=3)
df

,chisq,chisq_dimless,v1,v2,v3,n_scale,me/mu,mu/mt,s12^2,s13^2,s23^2,d/pi,r,m21^2,m3l^2,m1,m2,m3,eta1,eta2,J,Jmax,Sum(m_i),m_b,m_bb,nscale
0,4.641283e+04,4.640353e+04,-1.837044,2.321339,-0.501474,1,0.004672,1.000000,0.006463,0.043287,0.026156,0.0,0.031933,0.000077,0.002414,0.009606,0.013014,0.050060,0.0,0.0,0.000000e+00,0.002546,0.072680,0.014162,0.011412,0.009702
1,1.639647e+06,1.817613e+06,0.353166,0.108569,-0.360040,1,0.162005,1.000000,0.022900,0.580354,0.570825,1.0,0.501658,0.000666,0.001327,0.006096,0.026515,0.036939,1.0,0.0,2.898678e-18,0.023670,0.069550,0.034646,0.028905,0.015592
2,2.476548e+07,2.476542e+07,1.006576,0.686499,0.240679,1,1.000000,0.344542,0.532907,0.027556,0.075448,1.0,0.036890,0.000083,0.002258,0.013238,0.016079,0.049326,1.0,1.0,2.604978e-18,0.021271,0.078643,0.016820,0.015728,0.015394
3,2.476548e+07,2.476542e+07,1.006576,0.686499,0.240679,1,1.000000,0.344542,0.532907,0.027556,0.075448,1.0,0.036890,0.000083,0.002258,0.013238,0.016079,0.049326,1.0,1.0,2.604978e-18,0.021271,0.078643,0.016820,0.015728,0.015394
4,2.476845e+07,2.477337e+07,0.810205,0.418544,0.154431,1,1.000000,0.413169,0.792593,0.024325,0.147279,1.0,0.115236,0.000181,0.001575,0.010645,0.017170,0.041089,0.0,0.0,2.677633e-18,0.021865,0.068904,0.017216,0.016484,0.014220
5,2.477023e+07,2.477442e+07,0.819841,0.411874,0.154424,1,1.000000,0.419267,0.808184,0.022534,0.150425,1.0,0.109534,0.000174,0.001592,0.010832,0.017078,0.041341,0.0,0.0,2.529244e-18,0.020653,0.069251,0.017171,0.016500,0.014269
6,2.477023e+07,2.477444e+07,0.819638,0.411950,0.154015,1,1.000000,0.419521,0.807557,0.022621,0.150426,1.0,0.109648,0.000174,0.001591,0.010831,0.017082,0.041336,0.0,0.0,2.537044e-18,0.020717,0.069249,0.017175,0.016502,0.014269
7,2.477690e+07,2.483311e+07,0.544192,0.245367,0.029308,1,1.000000,0.546875,0.806279,0.041631,0.213546,1.0,0.296963,0.000409,0.001378,0.007957,0.021739,0.037968,1.0,0.0,3.878505e-18,0.031670,0.067664,0.021307,0.020067,0.014814
8,2.477753e+07,2.482018e+07,0.585751,0.242268,0.029161,1,1.000000,0.567453,0.836159,0.034361,0.220464,1.0,0.263308,0.000367,0.001394,0.008512,0.020966,0.038297,1.0,0.0,3.363579e-18,0.027466,0.067775,0.020722,0.019746,0.014707
9,2.477753e+07,2.482018e+07,0.585751,0.242268,0.029161,1,1.000000,0.567453,0.836159,0.034361,0.220464,1.0,0.263308,0.000367,0.001394,0.008512,0.020966,0.038297,1.0,0.0,3.363579e-18,0.027466,0.067775,0.020722,0.019746,0.014707


In [12]:
print(df.to_html(col_space=10))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th style="min-width: 10px;"></th>
      <th style="min-width: 10px;">chisq</th>
      <th style="min-width: 10px;">chisq_dimless</th>
      <th style="min-width: 10px;">v1</th>
      <th style="min-width: 10px;">v2</th>
      <th style="min-width: 10px;">v3</th>
      <th style="min-width: 10px;">n_scale</th>
      <th style="min-width: 10px;">me/mu</th>
      <th style="min-width: 10px;">mu/mt</th>
      <th style="min-width: 10px;">s12^2</th>
      <th style="min-width: 10px;">s13^2</th>
      <th style="min-width: 10px;">s23^2</th>
      <th style="min-width: 10px;">d/pi</th>
      <th style="min-width: 10px;">r</th>
      <th style="min-width: 10px;">m21^2</th>
      <th style="min-width: 10px;">m3l^2</th>
      <th style="min-width: 10px;">m1</th>
      <th style="min-width: 10px;">m2</th>
      <th style="min-width: 10px;">m3</th>
      <th style="min-width: 10px;">eta1</th>
      <th style=

Well, from the high value of $\chi^2$, we see that this model doesn't seem to be able to replicate the experimentally measured values. Let us take a look at the individual contributions to $\chi^2$ for the first point by

In [7]:
Model0.print_chisq(df.loc[0])

'me/mu': 0.004671873887378912,   chisq: 0.4104075183847923
'mu/mt': 1.0,   chisq: 43960.11111111111
's12^2': 0.006463068124372668,   chisq: 682.9674663229889
's13^2': 0.043286529978007846,   chisq: 1146.7998894965222
's23^2': 0.026155918753120917,   chisq: 597.7161072913841
'd/pi': 0.0,   chisq: 11.07305
'm21^2': 7.707858000157868e-05,   chisq: 1.9797798610722714
'm3l^2': 0.0024137253939102614,   chisq: 11.767751824690869
Total chi-square: 46412.82556342615


It looks like several observables are not in agreement with the experimental data. Note that $\chi^2=x$ is often interpreted as to the specific point lying in the $\sqrt{x}\,\sigma$ confidence level region.

All in all, the model was probably to simple or we needed to widen the boundaries of our parameter space.